###### Задание 1.

Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [124]:
KEYWORDS = ['python', 'парсинг']

In [471]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')
#posts

In [126]:
habr_news = pd.DataFrame()
for post in posts:
    i = 0
    for t1 in post('h2'):
        for t2 in t1('a'):
            if any(keyword in t2.text.lower() for keyword in KEYWORDS):
                i = 1
    if i == 0:
        for t1 in post('div', class_='post__body'):
            for t2 in t1('div', class_='post__text'):
                if any(keyword in t2.text.lower() for keyword in KEYWORDS):
                    i = 1
    if i == 1:
        for t1 in post('header'):
            for t2 in t1('span', class_='post__time'):
                date = t2.text
        for t1 in post('h2'):
            for t2 in t1('a'):
                title = t2.text
                ref = t2.attrs.get('href')
        row = {'date': date, 'title': title, 'reference': ref}
        habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
habr_news

,date,title,reference
0,сегодня в 13:16,"Пайплайны и частичное применения функций, заче...",https://habr.com/ru/post/517780/


Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [128]:
habr_news = pd.DataFrame()
for post in posts:
    for t1 in post('h2'):
        for t2 in t1('a'):
            ii = 0
            title = t2.text
            ref = t2.attrs.get('href')
            req1 = requests.get(ref)
            soup1 = BeautifulSoup(req1.text, 'html.parser')
            posts1 = soup1.find_all('article', class_='post')
            for post1 in posts1:
                for tt1 in post1('div', class_='post__wrapper'):
                    for tt2 in tt1('div', class_='post__body'):
                        for tt3 in tt2('div', class_='post__text'):
                            text = tt3.text
                            if any(keyword in tt3.text.lower() for keyword in KEYWORDS):
                                ii = 1
            if ii == 1:                    
                date = pd.to_datetime(soup1.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
                row = {'date': date, 'title': title, 'reference': ref, 'text': text}
                habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
habr_news

,date,title,reference,text
0,2020-09-04,Ускоряем Ansible,https://habr.com/ru/post/516028/,"\n\r\nНи для кого не секрет, что с настройками..."
0,2020-09-04,Как кибербезопасность трансформирует рынок ИТ ...,https://habr.com/ru/post/517860/,Несмотря на ежегодный рост бюджетов и разнообр...
0,2020-09-04,Заметки Дата Сайентиста: на что обратить внима...,https://habr.com/ru/company/ruvds/blog/517830/,\r\nМы снова в эфире и продолжаем цикл заметок...
0,2020-09-04,"Пайплайны и частичное применения функций, заче...",https://habr.com/ru/post/517780/,\nОдно из главных достоинств Python — его выра...
0,2020-09-04,Портируем утилиту командной строки с Go/Rust на D,https://habr.com/ru/post/517790/,"Несколько дней назад, на реддите в «программир..."


###### Задание 2.

Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [130]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [ ]:
import json

In [ ]:
headers= {
    'Accept': 'application/json',
    'Accept-Encoding': 'gzip',
    'Accept-Language': 'ru-RU,ru',
    'Connection': 'keep-alive',
    'Content-Length': '31',
    'Content-Type': 'application/json',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/hackcheck',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Version': '0'}

In [ ]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'


In [416]:
leakage = pd.DataFrame()
for em_ad in EMAIL:
    data={'emailAddresses': [em_ad]}
    response = requests.post(url, json = data, headers=headers)
    res = response.text
    json_obj = json.loads(res)
    for key, value in json_obj.items():
        if key == 'breaches':
            for k, val in value.items():
                for k1, v2 in val.items():
                    if  k1 == 'site': 
                        site = v2
                    elif k1 == 'description':
                        description = v2 
                    elif k1 == 'publishDate':
                        date = pd.to_datetime(v2, dayfirst=True).date()          
                row = {'email': em_ad, 'date': date, 'site': site, 'description': description}
                leakage = pd.concat([leakage, pd.DataFrame([row])])  
leakage  

,email,date,site,description
0,xxx@x.ru,2019-03-28,verifications.io,Big data e-mail verification platform verifica...
0,xxx@x.ru,2020-05-21,vk.com,"At some time in 2020, the Russian social netwo..."
0,xxx@x.ru,2017-02-14,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,xxx@x.ru,2016-10-29,vk.com,Popular Russian social networking platform VKo...
0,xxx@x.ru,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,xxx@x.ru,2020-09-10,na,This is a compilation of files including breac...
0,xxx@x.ru,2017-02-14,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,xxx@x.ru,2017-01-31,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,xxx@x.ru,2016-10-23,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,yyy@y.com,2019-03-28,verifications.io,Big data e-mail verification platform verifica...


Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [ ]:
from datetime import datetime

In [405]:
https://oauth.vk.com/authorize?client_id=удален&scope=photos,audio,video,docs,notes,pages,status,offers,questions,wall,groups,messages,email,notifications,stats,ads,offline,docs,pages,stars,notifications&response_type=token

In [406]:
https://oauth.vk.com/blank.html#access_token=удален&expires_in=86400&user_id=удален

In [406]:
https://api.vk.com/method/wall.get?user_id=удален&v=5.52
https://vk.com/wall-30159897_51072

In [423]:
TOKEN = 'удален'
VERSION = '5.52'
group_id = '-30159897'
SLEEP = 0.33

In [424]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'owner_id': group_id,
    'count': 50    
}

In [425]:
r = requests.get('https://api.vk.com/method/wall.get', params = params)

In [472]:
r.json()
re = r.json()
#re

In [470]:
posts = pd.DataFrame()
for list in re['response']['items']:    
    date1 = datetime.fromtimestamp(list['date'])
    text1 = list['text']
    row = {'date': date1, 'text': text1}
    posts = pd.concat([posts, pd.DataFrame([row])])  
posts  


,date,text
0,2020-09-07 11:00:10,Мы снова дарим подарки 🎁\n \nНа этот раз всё с...
0,2020-09-11 10:18:00,Начинать что-то новое необязательно с понедель...
0,2020-09-10 17:50:00,"Когда работы много, а отдыха почти нет, приход..."
0,2020-09-10 10:42:00,"Бывает заходишь на сайт: вроде все красиво, но..."
0,2020-09-09 17:20:00,Кем работать в интернет-маркетинге? Собрали са...
0,2020-09-09 10:12:00,Сегодня загадка на внимательность: давайте пом...
0,2020-09-08 18:12:00,А давайте помечтаем: кем бы вы хотели стать че...
0,2020-09-08 13:00:10,В сентябре внутри каждого непроизвольно возник...
0,2020-09-08 10:42:00,Учёба не всегда приходит с началом сентября. М...
0,2020-09-07 18:42:00,"Бывает, весь день занимаешься рабочими делами,..."
